In [ ]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time

In [5]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'a146a2e813e29e01e0efe8746e676354b1a03a4ed410442b3101f6f7ffb1dcf6:856cd761f8ed49a5a24b540008c89918'

## User Input Class

In [6]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = self.passwordHash
            )
        userTable.save()
        

In [7]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123'})
example

UserInput(username='DelG', password='123')

In [24]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    @staticmethod
    @beartype
    def parseInput(event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise ParseInputError(e)
 
    def saveUserMethod(user:UserInput)->bool:
        try:
            user.saveTable()
            return True
        except Exception as e:
            raise e
    
        

## Main Lambda Handler

In [25]:
#export
def signUp(events, *args):
    try:
        user = H.parseInput(event)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    # except Exception as e:
    #     return Response.returnError(f' unknown error {e}')
    

In [26]:
input_ = {'username': 'DelG', 'password': '123'}
event = Event.getInput(body = input_)
signUp(event)

AttributeError: As of v1.0 PyanmoDB Models must have a table_name
Model: src.userTable.UserTable
See https://pynamodb.readthedocs.io/en/latest/release_notes.html

In [37]:
epoch = time()
epoch

1658677547.060287